# Caso Práctico: Neurona de McCulloch y Pitts

## 2. Aplicando la MPNeuron a un caso práctico real

### 2.1. Conjunto de datos

Esta es una copia de los conjuntos de datos de UCI ML Breast Cancer Wisconsin (Diagnóstico). https://goo.gl/U2Uwz2

Las características de entrada se calculan a partir de una imagen digitalizada de un aspirado de aguja fina (FNA) de una masa mamaria. Describen las características de los núcleos celulares presentes en la imagen.

El plano de separación descrito anteriormente se obtuvo utilizando el método de árbol de múltiples superficies (MSM-T) [K. P. Bennett, "Construcción de un árbol de decisión mediante programación lineal". Proceedings of the 4th Midwest Artificial Intelligence and Cognitive Science Society, pp. 97-101, 1992], un método de clasificación que utiliza la programación lineal para construir un árbol de decisión. Los rasgos relevantes se seleccionaron mediante una búsqueda exhaustiva en el espacio de 1-4 rasgos y 1-3 planos de separación.

El programa lineal real utilizado para obtener el plano de separación en el espacio tridimensional es el que se describe en: [K. P. Bennett y O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

Esta base de datos también está disponible a través del servidor ftp UW CS:

ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

### Referencias

* W.N. Street, W.H. Wolberg and O.L. Mangasarian. Nuclear feature extraction for breast tumor diagnosis. IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science and Technology, volume 1905, pages 861-870, San Jose, CA, 1993.
* O.L. Mangasarian, W.N. Street and W.H. Wolberg. Breast cancer diagnosis and prognosis via linear programming. Operations Research, 43(4), pages 570-577, July-August 1995.
* W.H. Wolberg, W.N. Street, and O.L. Mangasarian. Machine learning techniques to diagnose breast cancer from fine-needle aspirates. Cancer Letters 77 (1994) 163-171.

### 2.3. Lectura del conjunto de datos

In [ ]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
X = breast_cancer.data
Y = breast_cancer.target

In [ ]:
dir(breast_cancer)

In [ ]:
X

### 2.2. Visualización del conjunto de datos

In [ ]:
import pandas as pd
df = pd.DataFrame(X, columns=breast_cancer.feature_names)
df

In [ ]:
Y

### 2.3. División del conjunto de datos

In [ ]:
from sklearn.model_selection import train_test_split #divide datos de entrenamiento y el test
X_train, X_test, y_train, y_test = train_test_split(df, Y, stratify=Y)
print("Tamaño del conjunto de datos de entrenamiento: ", len(X_train))
print("Tamaño del conjunto de datos de pruebas: ", len(X_test))

### 2.4. Implementación de una MPNeuron más avanzada

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score # para ver la exactitud del modelo
class MPNeuron:
    def __init__(self):
        self.threshold = None
    def model(self, x): # x es [1,0,0,1]
        return (sum(x) >= self.threshold) # suma si los 1 son mayor que el umbral
    def predict(self, X): # X es un array de arrays[[1,0,0,0],[1,0,0]]
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result) # [True, False, False, True]
        return np.array(Y)
    def fit(self, X, Y): # buscar un threshold óptimo dinámicamente
        accuracy = {}
        # Seleccionamos un threshold entre el # de características de entrada
        for th in range(X.shape[1] + 1): # .shape[0] es nº filas y .shape[1] es nº columnas
            self.threshold = th # voy probando threshold  hasta dar con el mas exacto
            Y_pred = self.predict(X)
            accuracy[th] = accuracy_score(Y_pred, Y) #diferencia entre predicion y Y ya etiquetados (realidad)
        # Seleccionamos el threshold que mejores resultados proporciona
        self.threshold = max(accuracy, key=accuracy.get)

Seguimos teniendo un problema debido a que en nuestro conjunto de datos las características de entrada reciben valores continuos, sin embargo, nuestra MPNeuron solo procesa características de entrada con valor binario

In [ ]:
# Para transformar un valor a binario
import matplotlib.pyplot as plt
print(pd.cut([0.04, 2, 4, 5, 6, 0.02, 0.6], bins=2, labels=[0, 1]))
plt.hist([0.04, 0.3, 4, 5, 6, 0.02, 0.6], bins=2) 
plt.show()

In [ ]:
# Transformamos las caracteríticas de entrada a un valor binario
X_train_bin = X_train.apply(pd.cut, bins=2, labels=[1, 0]) #valores corte para convertir en 0/1
X_test_bin = X_test.apply(pd.cut, bins=2, labels=[1, 0])
X_train_bin

In [ ]:
mp_neuron = MPNeuron() # Instanciamos el modelo MPNeuron
mp_neuron.fit(X_train_bin.to_numpy(), y_train) # Encontramos el threshold óptimo

In [ ]:
# Threshold óptimo seleccionado
mp_neuron.threshold

In [ ]:
# Realizamos predicciones para ejemplos nuevos que no se encuentran en el conjunto de datos de entrenamiento
Y_pred = mp_neuron.predict(X_test_bin.to_numpy())

In [ ]:
Y_pred

In [ ]:
accuracy_score(y_test, Y_pred) # Calculamos la exactitud de nuestra predicción
# y_test son las soluciones etiquetadas (correctas) y Y_pred es la prediccion de mi modelo con los datos de test una vez ya entrenado el modelo -> acierta con un 81% de exactitud

In [ ]:
# Calculamos la matriz de confusión -> verdadedos positivos/negativos y falsos positivos/negativos
from sklearn.metrics import confusion_matrix # calcula la matriz de confusion
confusion_matrix(y_test, Y_pred)